In [7]:
# ======================================================
# train_model.py
# Fraud Detection Pipeline
# ======================================================

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import classification_report, roc_auc_score, f1_score, average_precision_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import SMOTE

# ======================================================
# 1. LOAD DATA & PREPROCESSING
# ======================================================
df = pd.read_csv("creditcard.csv")
df = df.dropna(subset=["Class"])

# Log transformation to handle skewed 'Amount' data
df['log_Amount'] = np.log1p(df['Amount'])

# Drop unnecessary columns
X = df.drop(["Class", "Time", "Amount"], axis=1)
y = df["Class"]

print(f"Dataset shape: {df.shape}")
print(f"Fraud ratio: {y.mean():.5f}")

# ======================================================
# 2. TRAIN / TEST SPLIT
# ======================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# ======================================================
# 3. SCALING
# ======================================================
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ======================================================
# 4. HANDLE CLASS IMBALANCE (SMOTE)
# ======================================================
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train_scaled, y_train)

print(f"Training set shape after SMOTE: {X_train_sm.shape}")

# ======================================================
# 5. MODEL DEFINITIONS (OPTIMIZED)
# ======================================================
models = {
    "Logistic Regression": LogisticRegression(
        max_iter=1000, 
        solver="liblinear",
        C=0.01
    ),
    "Decision Tree": DecisionTreeClassifier(
        random_state=42,
        max_depth=5,
        min_samples_leaf=5
    ),
   "Random Forest": RandomForestClassifier(
        random_state=42,
        n_estimators=100,    
        max_depth=20,       
        min_samples_leaf=1,  
        n_jobs=-1,
        verbose=2      
    )
}

# ======================================================
# 6. TRAINING & EVALUATION
# ======================================================
for name, model in models.items():
    print(f"\n==============================")
    print(f"Training {name}")
    print(f"==============================")

    model.fit(X_train_sm, y_train_sm)

    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]

    print(classification_report(y_test, y_pred))
    print(f"F1-score: {f1_score(y_test, y_pred):.4f}")
    print(f"ROC-AUC:  {roc_auc_score(y_test, y_proba):.4f}")
    print(f"PR-AUC:   {average_precision_score(y_test, y_proba):.4f}")

Dataset shape: (284807, 32)
Fraud ratio: 0.00173
Training set shape after SMOTE: (454902, 29)

Training Logistic Regression
              precision    recall  f1-score   support

           0       1.00      0.97      0.99     56864
           1       0.06      0.92      0.11        98

    accuracy                           0.97     56962
   macro avg       0.53      0.95      0.55     56962
weighted avg       1.00      0.97      0.99     56962

F1-score: 0.1100
ROC-AUC:  0.9702
PR-AUC:   0.7192

Training Decision Tree
              precision    recall  f1-score   support

           0       1.00      0.96      0.98     56864
           1       0.04      0.89      0.07        98

    accuracy                           0.96     56962
   macro avg       0.52      0.92      0.53     56962
weighted avg       1.00      0.96      0.98     56962

F1-score: 0.0708
ROC-AUC:  0.9656
PR-AUC:   0.4126

Training Random Forest


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   39.9s


building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100building tree 56 of 100

building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.5min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.74      0.84      0.78        98

    accuracy                           1.00     56962
   macro avg       0.87      0.92      0.89     56962
weighted avg       1.00      1.00      1.00     56962

F1-score: 0.7847
ROC-AUC:  0.9659
PR-AUC:   0.8723


[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
